In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
import scipy.special as special
import sys
import h5py

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_m = 0.315 # current matter abundance
omega_lambda = 0.685 # cosmological constant
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium fraction
f_H = 0.76 # hydrogen fraction
nu_0 = 2.47e15 * 4 / 3 # center frequency of Lyman-alpha emission, in [Hz]
C = 29979245800.0 # speed of light, in [cm s^-1]
h = 6.6261e-27 # planck constant, in [cm^2 g s^-1]
k = 1.380649e-16 # Boltzmann constant, in [g cm^2 s^-2 K^-1]
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_H = 1.6735575e-24 # hydrogen atom mass in g

T_bb = 47834.5 # 21cmFAST setting blackbody temperature in K

In [17]:
"""
To use this class, call front_velocity(z, T_bb, gamma_12, delta_m)

z is redshift, T_bb is blackbody temperature in K, gamma_12 is the photoionization rate in unit cgs*1e12,
delta_m is the matter overdensity in cgs unit

all variables should be individual values rather than array or list

.nH() will return to the number density of neutral hydrogen atom in cm^-3
.U() will return to the front velocity in cm/s
"""

class front_velocity:
    def __init__(self, z, T_bb, gamma_12, delta_m):
        gamma_12 = gamma_12 * 1e-12 # convert to cgs unit
        self.z = z
        self.T_bb = T_bb
        self.gamma_12 = gamma_12
        self.delta_m = delta_m
        
    def sigma_HI(self, dnu):
        sigma_0 = 6.3e-18 # cm^2
        nu = dnu + nu_0
        epsilon = np.sqrt(nu / nu_0 - 1)
        sigma = sigma_0 * (nu_0 / nu)**4 * np.exp(4 - 4 * np.arctan(epsilon) / epsilon) / (1 - np.exp(-2 * np.pi / epsilon))
        return sigma
    
    def F_nu(self, dnu, T_bb):
        nu = dnu + nu_0
        f_nu = 2 * h * nu**3 / C**2 / (np.exp(h * nu / k / T_bb) - 1)/ h / nu
        return f_nu

    def int_F(self, T):
        lowerlimit = (2.17872e-11 / h) - nu_0
        upperlimit = (4 * 2.17872e-11 / h) - nu_0
        int_f = integrate.quad(self.F_nu, lowerlimit, upperlimit, args=T)
        return int_f

    def H_F(self, dnu, T_bb):
        sigma_0 = 6.3e-18 # cm^2
        nu = dnu + nu_0
        epsilon = np.sqrt(nu / nu_0 - 1)
        sigma = sigma_0 * (nu_0 / nu)**4 * np.exp(4 - 4 * np.arctan(epsilon) / epsilon) / (1 - np.exp(-2 * np.pi / epsilon))
        ex = np.float64(h * nu / k / T_bb) # if ex larger than 700, should use float128
        f_nu = 2 * h * nu**3 / C**2 / (np.exp(ex) - 1) / h / nu
        return sigma * f_nu

    def int_Fsigma(self, T):
        lowerlimit = (2.17872e-11 / h) - nu_0 # 2.17872e-11 is 13.6eV in cgs unit
        upperlimit = (4 * 2.17872e-11 / h) - nu_0
        int_F = integrate.quad(self.H_F, lowerlimit, upperlimit, args=T)
        return int_F

    def sigma_avg(self, T):
        Fsig = self.int_Fsigma(T)
        F = self.int_F(T)
        sigma = Fsig[0] / F[0]
        return sigma
        
    def nH(self):
        omega_b = omega_b_0 * (1 + self.z)**3
        rho_crit = 3 * H_r * H_r / 8 / np.pi / G
        return omega_b * f_H * rho_crit / m_H * (1 + self.delta_m)
        
    def U(self):
        sigma_H_avg = self.sigma_avg(self.T_bb)
        nH = self.nH()
        return self.gamma_12 / sigma_H_avg / (nH * (1 + f_He) + self.gamma_12 / sigma_H_avg / C)

In [19]:
"""
Test section
"""
def load_binary_data(filename, dtype=np.float32):
    """
    We assume that the data was written
    with write_binary_data() (little endian).
    """
    f = open(filename, "rb")
    data = f.read()
    f.close()
    _data = np.frombuffer(data, dtype)
    if sys.byteorder == 'big':
        _data = _data.byteswap()
    return _data

file_gamma_12 = r'C:\data\Gamma12aveHII_z008.11_HIIfilter1_RHIImax50_200_300Mpc' # direction of gamma_12
file_delta_m = r'C:\data\updated_smoothed_deltax_z008.11_200_300Mpc' # direction of delta_m

gamma_12_origin = load_binary_data(file_gamma_12, dtype=np.float32) # photoionization rate
delta_m_origin = load_binary_data(file_delta_m, dtype=np.float32) # matter overdensity

z = 8.11
data = front_velocity(z, T_bb, gamma_12_origin[9], delta_m_origin[9])

print(data.nH())
print(data.U())

0.0005305083655723939
21233921.24795486
